In [1]:
%reload_ext dotenv
%dotenv .env

In [2]:
import os
import re

from mechanicalsoup import StatefulBrowser

browser = StatefulBrowser()

#### Get all tram line timetable URLs

In [3]:
browser.open(os.environ.get('TIMETABLE_URL'))

<Response [200]>

In [4]:
tram_lines = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("a", class_=lambda x: x and x.startswith("linia"))
    if len(item.text.strip()) <= 2
}

tram_lines

{'1': '?lang=PL&linia=1',
 '3': '?lang=PL&linia=3',
 '4': '?lang=PL&linia=4',
 '5': '?lang=PL&linia=5',
 '8': '?lang=PL&linia=8',
 '9': '?lang=PL&linia=9',
 '10': '?lang=PL&linia=10',
 '11': '?lang=PL&linia=11',
 '13': '?lang=PL&linia=13',
 '14': '?lang=PL&linia=14',
 '16': '?lang=PL&linia=16',
 '17': '?lang=PL&linia=17',
 '18': '?lang=PL&linia=18',
 '19': '?lang=PL&linia=19',
 '20': '?lang=PL&linia=20',
 '21': '?lang=PL&linia=21',
 '22': '?lang=PL&linia=22',
 '24': '?lang=PL&linia=24',
 '44': '?lang=PL&linia=44',
 '49': '?lang=PL&linia=49',
 '50': '?lang=PL&linia=50',
 '52': '?lang=PL&linia=52',
 '62': '?lang=PL&linia=62',
 '64': '?lang=PL&linia=64',
 '69': '?lang=PL&linia=69',
 '72': '?lang=PL&linia=72'}

## Line 1

#### Line variants

In [5]:
browser.open(os.environ.get('TIMETABLE_URL') + tram_lines['1'])

<Response [200]>

In [6]:
variant_urls = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("table")[7].findAll("a")
}

variant_urls

{'Elektromontaż - Cichy Kącik': '?lang=PL&akcja=index&rozklad=20240506&linia=1__1',
 'Cichy Kącik - Elektromontaż': '?lang=PL&akcja=index&rozklad=20240506&linia=1__2'}

#### First variant's stops

In [7]:
browser.open(os.environ.get('TIMETABLE_URL') + list(variant_urls.values())[0])

<Response [200]>

In [8]:
stop_urls = {
    item.text.strip(): item.find("a") and item.find("a").get("href")[1:]
    for item in browser.page.findAll("table")[10].findAll("td")
    if item.text.strip()
}

stop_urls

{'Elektromontaż 02': '?lang=PL&rozklad=20240506&linia=1__1__1',
 'Grodzki Urząd Pracy 02': '?lang=PL&rozklad=20240506&linia=1__1__2',
 'Wańkowicza 01': '?lang=PL&rozklad=20240506&linia=1__1__3',
 'Cienista 01': '?lang=PL&rozklad=20240506&linia=1__1__4',
 'Teatr Ludowy 01': '?lang=PL&rozklad=20240506&linia=1__1__5',
 'Rondo Kocmyrzowskie im. Ks. Gorzelanego 02': '?lang=PL&rozklad=20240506&linia=1__1__6',
 'Bieńczycka 01': '?lang=PL&rozklad=20240506&linia=1__1__7',
 'Rondo Czyżyńskie 04': '?lang=PL&rozklad=20240506&linia=1__1__8',
 'Centralna 02': '?lang=PL&rozklad=20240506&linia=1__1__9',
 'Rondo 308. Dywizjonu 02': '?lang=PL&rozklad=20240506&linia=1__1__10',
 'Ogród Doświadczeń 02': '?lang=PL&rozklad=20240506&linia=1__1__11',
 'TAURON Arena Kraków al. Pokoju 02': '?lang=PL&rozklad=20240506&linia=1__1__12',
 'Dąbie 02': '?lang=PL&rozklad=20240506&linia=1__1__13',
 'Ofiar Dąbia 02': '?lang=PL&rozklad=20240506&linia=1__1__14',
 'Fabryczna 02': '?lang=PL&rozklad=20240506&linia=1__1__15',
 

#### First stop's departures

In [9]:
browser.open(os.environ.get('TIMETABLE_URL') + list(stop_urls.values())[0])

<Response [200]>

In [10]:
for item in browser.page.findAll("table")[13].findAll("tr")[1:-2]:
    cells = item.findAll("td")

    hour = int(cells[0].text)
    weekday_minutes = [int(minute) for minute in cells[1].text.strip().split()]
    saturday_minutes = [int(minute) for minute in cells[2].text.strip().split()]
    sunday_minutes = [int(minute) for minute in cells[3].text.strip().split()]

    print(hour, weekday_minutes, saturday_minutes, sunday_minutes, sep="\t")

3	[]	[]	[]
4	[44, 57]	[50]	[57]
5	[11, 28, 43, 59]	[10, 30, 50]	[27, 57]
6	[13, 21, 28, 34, 43, 51, 58]	[10, 30, 50]	[27, 57]
7	[6, 13, 21, 28, 36, 43, 51, 58]	[10, 30, 50]	[27, 59]
8	[6, 13, 21, 28, 36, 43, 51, 58]	[9, 29, 49]	[29, 59]
9	[6, 13, 21, 28, 36, 43, 51, 58]	[9, 29, 49]	[29, 49]
10	[6, 13, 21, 28, 36, 43, 51, 58]	[9, 29, 49]	[9, 29, 49]
11	[6, 13, 21, 28, 36, 43, 51, 58]	[9, 29, 49]	[9, 29, 49]
12	[6, 13, 21, 28, 36, 43, 51, 58]	[9, 29, 49]	[9, 29, 49]
13	[6, 13, 21, 28, 36, 43, 51]	[9, 29, 49]	[9, 29, 49]
14	[0, 8, 17, 24, 32, 40, 47, 55]	[9, 29, 49]	[9, 29, 49]
15	[2, 10, 17, 24, 32, 39, 47, 54]	[9, 29, 49]	[9, 29, 49]
16	[2, 9, 17, 24, 32, 39, 47, 54]	[9, 29, 49]	[9, 29, 49]
17	[2, 9, 17, 24, 32, 39, 47, 54]	[9, 29, 49]	[9, 29, 49]
18	[2, 9, 17, 24, 32, 39, 49, 58]	[9, 29, 49]	[9, 29, 49]
19	[6, 14, 25, 36, 53]	[9, 29, 49]	[9, 29, 49]
20	[6, 19, 34, 53]	[10, 30, 50]	[10, 30, 50]
21	[9, 28, 47]	[10, 30, 50]	[10, 30, 50]
22	[9, 29, 49]	[10, 30]	[10, 30]
23	[]	[3]	[3]
0	[]	

## Line 50

#### Line variants

In [11]:
browser.open(os.environ.get('TIMETABLE_URL') + tram_lines['50'])

<Response [200]>

In [12]:
variant_urls = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("table")[7].findAll("a")
}

variant_urls

{'Górka Narodowa P+R - Borek Fałęcki': '?lang=PL&akcja=index&rozklad=20240506&linia=50__1',
 'Borek Fałęcki - Górka Narodowa P+R': '?lang=PL&akcja=index&rozklad=20240506&linia=50__2'}

#### First variant's stops

In [13]:
browser.open(os.environ.get('TIMETABLE_URL') + list(variant_urls.values())[0])

<Response [200]>

In [14]:
stop_urls = {
    item.text.strip(): item.find("a") and item.find("a").get("href")[1:]
    for item in browser.page.findAll("table")[10].findAll("td")
    if item.text.strip()
}

stop_urls

{'Górka Narodowa P+R 01': '?lang=PL&rozklad=20240506&linia=50__1__1',
 'Papierni Prądnickich 02': '?lang=PL&rozklad=20240506&linia=50__1__2',
 'Kuźnicy Kołłątajowskiej 02': '?lang=PL&rozklad=20240506&linia=50__1__3',
 'Bociana 02': '?lang=PL&rozklad=20240506&linia=50__1__4',
 'Siewna Wiadukt 02': '?lang=PL&rozklad=20240506&linia=50__1__5',
 'Górnickiego 01': '?lang=PL&rozklad=20240506&linia=50__1__6',
 'Białoprądnicka 01': '?lang=PL&rozklad=20240506&linia=50__1__7',
 'Pachońskiego P+R 02': '?lang=PL&rozklad=20240506&linia=50__1__8',
 'Krowodrza Górka P+R 02': '?lang=PL&rozklad=20240506&linia=50__1__9',
 'Bratysławska 02': '?lang=PL&rozklad=20240506&linia=50__1__10',
 'Szpital Narutowicza 02': '?lang=PL&rozklad=20240506&linia=50__1__11',
 'Dworzec Towarowy 02': '?lang=PL&rozklad=20240506&linia=50__1__12',
 'Politechnika 04': '?lang=PL&rozklad=20240506&linia=50__1__13',
 'Dworzec Główny Tunel 01': '?lang=PL&rozklad=20240506&linia=50__1__14',
 'Rondo Mogilskie 03': '?lang=PL&rozklad=20240

#### First stop's departures

In [15]:
browser.open(os.environ.get('TIMETABLE_URL') + list(stop_urls.values())[0])

<Response [200]>

In [16]:
for item in browser.page.findAll("table")[13].findAll("tr")[1:-2]:
    cells = item.findAll("td")

    hour = int(cells[0].text)
    print()
    weekday_minutes = [int(re.sub("[^0-9]", "", minute)) for minute in cells[1].text.strip().split()]
    saturday_minutes = [int(re.sub("[^0-9]", "", minute)) for minute in cells[2].text.strip().split()]
    sunday_minutes = [int(re.sub("[^0-9]", "", minute)) for minute in cells[3].text.strip().split()]

    print(hour, weekday_minutes, saturday_minutes, sunday_minutes, sep="\t")


3	[]	[]	[]

4	[43, 57]	[39, 59]	[44]

5	[12, 23, 28, 33, 38, 43, 48, 53, 58]	[19, 39, 59]	[14, 44]

6	[3, 8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58]	[19, 39, 59]	[14, 44]

7	[3, 8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58]	[19, 39, 56]	[14, 39, 59]

8	[3, 8, 13, 18, 24, 29, 34, 39, 46, 54]	[6, 16, 26, 36, 46, 56]	[19, 39, 59]

9	[1, 9, 16, 24, 31, 39, 46, 54]	[6, 16, 26, 36, 46, 56]	[19, 39, 59]

10	[1, 9, 16, 24, 31, 39, 46, 54]	[6, 16, 26, 36, 46, 56]	[19, 39, 59]

11	[1, 9, 16, 24, 31, 39, 46, 54]	[6, 16, 26, 36, 46, 56]	[19, 39, 59]

12	[1, 9, 16, 24, 31, 39, 46, 54]	[6, 16, 26, 36, 46, 56]	[19, 39, 59]

13	[1, 9, 16, 24, 31, 36, 41, 46, 51, 56]	[6, 16, 26, 36, 46, 56]	[19, 39, 56]

14	[1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56]	[6, 16, 26, 36, 46, 56]	[6, 16, 26, 36, 46, 56]

15	[1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56]	[6, 16, 26, 36, 46, 56]	[6, 16, 26, 36, 46, 56]

16	[1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56]	[6, 16, 26, 36, 46, 56]	[6, 16, 26, 36, 46, 56]

17	[1, 6